In [4]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } },
    "HTML-CSS": {
        availableFonts: [],
        preferredFont: null, 
        scale: 100,
        minScaleAdjust: 100,
        webFont: "STIX-Web"
    }
});

<IPython.core.display.Javascript object>

In [5]:
%%html
<style>
div.text_cell_render {
    font-size: 14pt;
    line-height: 22pt;
}
</style>

# Model description

__Notes:__

- ${}^\text{T}$ represents transpose.
- Elements in __bold__ are matrices
- Multiplication between matrices is always matrix multiplication, not
  element-wise

Species $i$ has a $1 \times q$ matrix of traits ($\mathbf{V}_i$), and
its per-capita growth (equivalent to fitness) is

\begin{equation} \label{eq:fitness}
\begin{split}
    F_{i} &= \exp \left\{
        r_0 - f ~ \mathbf{V}_i ~ \mathbf{C} ~ \mathbf{V}_{i}^{\text{T}} -
        \alpha_0 ~ \text{e}^{- \mathbf{V}_i \mathbf{V}_i^{\text{T}} } ~
        \mathbf{\Omega}_i
    \right\} \\
    \mathbf{\Omega}_i &\equiv N_{i} +
        \sum_{j \ne i}^{n}{ N_{j} ~ \text{e}^{
                -d \mathbf{V}_{j} \mathbf{V}_{j}^{\text{T}}} }
    \text{,}
\end{split}
\end{equation}

where...

- $N_i$ is the population density of species $i$
- $r_0$ is the baseline growth rate
- $f$ is the cost of diverging traits on the growth rate
- $\alpha_0$ is the base density dependence

Parameter $d$ determines how the evolution of traits in one species affects
competition experienced by others.
The $q \times q$ matrix $\mathbf{C}$ contains ones in diagonal elements, and the
non-additive trade-off of having multiple diverging traits ($\eta$) in off-diagonal
elements.
When $\eta > 0$, diverging multiple traits incurs an extra cost.


Traits at time $t+1$ are

\begin{equation} \label{eq:trait-change}
    \mathbf{V}_{i,t+1} = \mathbf{V}_{i,t} + \left( \frac{1}{F_i}
        \frac{\partial F_i}{\partial \mathbf{V}_{i}} \right) \sigma^2_i
    \text{,}
\end{equation}

where $\sigma^2_i$ is the additive genetic variance.




# Matrix solutions


The partial derivative of fitness in relation to traits for species $i$ is

$$
\frac{ \partial F_i }{ \partial \mathbf{V}_i } =
        \exp \left\{
            r_0 - \alpha_0 ~ \mathbf{\Gamma} ~
                \text{e}^{- \mathbf{V}_i \mathbf{V_i}^{\text{T}}}
            - f ~ \mathbf{V}_i ~ \mathbf{C} ~ \mathbf{V}_{i}^{\text{T}}
        \right\}
        \left[
            2 ~ \alpha_0 ~ \mathbf{\Gamma} ~
                \text{e}^{- \mathbf{V}_i \mathbf{V_i}^{\text{T}}}
                \mathbf{V}_i
            - f ~ \mathbf{V}_i (\mathbf{C} + \mathbf{C}^{\text{T}})
        \right]
    \text{.}
$$

Combining above with equation \ref{eq:trait-change} and with the fact that $\mathbf{C}$ is symmetrical, we find that trait values at time $t+1$ are

\begin{equation} \label{eq:trait-change-full}
    \mathbf{V}_{i,t+1} = \mathbf{V}_{i,t} + 2 ~ \sigma_i^2
    \left[
        \alpha_0 ~ \mathbf{\Gamma}_t ~
            \text{e}^{-\mathbf{V}_{i,t} \mathbf{V}_{i,t}^\text{T}} ~ \mathbf{V}_{i,t}
        - f ~ \mathbf{V}_{i,t} \mathbf{C}
    \right]
    \text{.}
\end{equation}

The partial derivative of traits at time $t+1$ with respect to traits at time $t$ is

\begin{equation} \label{eq:deriv-traits-i}
    \frac{ \partial \mathbf{V}_{i,t+1} }{ \partial \mathbf{V}_{i,t} } = \mathbf{I} + 2 ~ \sigma_i^2 ~
        \left[
            \alpha_0 ~ \mathbf{\Gamma}_t ~ \text{e}^{ - \mathbf{V}_{i,t} \mathbf{V}_{i,t}^{\text{T}} }
            \left(
                -2 ~ \mathbf{V}_{i,t}^{\text{T}} \mathbf{V}_{i,t} + \mathbf{I}
            \right) -
            f \mathbf{C}
        \right]
    \text{,}
\end{equation}

where $\mathbf{I}$ is a $q \times q$ identity matrix.
The matrices $\partial \mathbf{V}_{i,t+1} / \partial \mathbf{V}_{i,t}$ for $i \in \{1, \ldots, n\}$ represent the on-diagonal $q \times q$ blocks in the Jacobian matrix.
The off-diagonal blocks are $\partial \mathbf{V}_{i,t+1} / \partial \mathbf{V}_{k,t}$ where $k \ne i$.
To calculate this, it's useful to rearrange equation \ref{eq:trait-change-full} and extract the portion that includes $\mathbf{V}_{k,t}$:

$$
\begin{split}
    \mathbf{V}_{i,t+1} &= \mathbf{V}_{i,t} + 2 ~ \sigma_i^2
    \left[
        \left(
            N_{k,t} ~ \text{e}^{-d \mathbf{V}_{k,t} \mathbf{V}_{k,t}^\text{T}} +
            \mathbf{\Phi}_t
        \right)
        \left(
            \alpha_0 ~ \text{e}^{-\mathbf{V}_{i,t}
            \mathbf{V}_{i,t}^\text{T}} ~ \mathbf{V}_{i,t}
        \right)
        - f \mathbf{V}_{i,t} \mathbf{C}
    \right] \\
    \mathbf{\Phi}_t &= N_{i,t} + \sum_{j \ne i, j \ne k}^{n}{
        N_{j,t} ~ \text{e}^{-d \mathbf{V}_{j,t}
        \mathbf{V}_{j,t}^{\text{T}}} }
    \text{.}
\end{split}
$$

From this we calculated the partial derivative of $\mathbf{V}_{i,t+1}$ in relation to $\mathbf{V}_{k,t}$

\begin{equation} \label{eq:deriv-traits-k}
    \frac{ \partial\mathbf{V}_{i,t+1} }{ \partial\mathbf{V}_{k,t}} =
        -4 ~ d ~ \alpha_0 ~ \sigma^2 ~ N_{k,t}
        \left[
            \mathbf{V}_{k,t}^{\text{T}}
            \text{e}^{
                - \mathbf{V}_{i,t} \mathbf{V}_{i,t}^{\text{T}}
                - d \mathbf{V}_{k,t} \mathbf{V}_{k,t}^{\text{T}}
            }
            \mathbf{V}_{i,t}
        \right]
    \text{.}
\end{equation}



# Two-trait solutions


For these solutions, we will no longer use matrix multiplication.
Also note that since all traits have the same costs, benefits, and
non-additive effects, solutions for trait 1 and 2 are the same.
Only solutions for trait 1 are presented.

## Trait values

The two traits for species $i$ change as follows:

$$
\begin{split}
    V_{i1,t+1} &= V_{i1,t} + 2 ~ \sigma^2
    \left[
        \alpha_0 ~ \Gamma_t ~
            \text{e}^{-V_{i1,t}^2 - V_{i2,t}^2} ~ V_{i1,t}
        - f ~ ( V_{i1,t} + \eta ~ V_{i2,t} )
    \right] \\
    V_{i2,t+1} &= V_{i2,t} + 2 ~ \sigma^2
    \left[
        \alpha_0 ~ \Gamma_t ~
            \text{e}^{-V_{i2,t}^2 - V_{i1,t}^2} ~ V_{i2,t}
        - f ~ ( V_{i2,t} + \eta ~ V_{i1,t} )
    \right] \\
    \Gamma_t &\equiv N_{i,t} +
        \sum_{j \ne i}^{n}{ N_{j,t} ~ \text{e}^{
                -d (V_{j2,t}^2 + V_{j1,t}^2) } }
    \text{.}
\end{split}
$$

We'll now drop indices for species and time because we are
first focusing on the summed abundances ($\Gamma$)
and only using values for time $t$.


At equilibrium (assuming that $\sigma > 0$),

\begin{equation}
\begin{split}
    0 &= \alpha_0 ~ \Gamma ~
            \text{e}^{-V_{1}^2 - V_{2}^2} ~ V_{1}
        - f ~ ( V_{1} + \eta ~ V_{2} ) \\
    0 &=
        \alpha_0 ~ \Gamma ~
            \text{e}^{-V_{2}^2 - V_{1}^2} ~ V_{2}
        - f ~ ( V_{2} + \eta ~ V_{1} )
    \text{.}
\end{split}
\label{eq:two-traits-V-eq1}
\end{equation}


Thus, we have

$$
\begin{split}
    \alpha_0 ~ \Gamma ~ \text{e}^{-V_{1}^2 - V_{2}^2} &=
        \frac{ f ~ ( V_{1} + \eta ~ V_{2} ) }{ V_{1} } \\
    \alpha_0 ~ \Gamma ~ \text{e}^{-V_{1}^2 - V_{2}^2} &=
        \frac{ f ~ ( V_{2} + \eta ~ V_{1} ) }{ V_{2} }
    \text{.}
\end{split}
$$


Combining these leads to

$$
\begin{split}
    f ~ ( V_{1} + \eta ~ V_{2} ) ~ V_{2} &=
        f ~ ( V_{2} + \eta ~ V_{1} ) ~ V_{1} \\
    f V_{1} V_{2} + \eta V_{2}^2 &=
        f V_{1} V_{2} + \eta V_{1}^2 \\
    \eta V_{2}^2 &= \eta V_{1}^2 \\
    V_{1} &= \pm V_{2}
    \text{.}
\end{split}
$$

Thus, we have three scenarios of how $V_{1}$ relates to $V_{2}$:
$V_{1} = V_{2}$, $V_{1} = - V_{2}$, and
neither (when $\eta = 0$).
Plugging in the first of these scenarios
($V_{1} = V_{2}$) into equation \ref{eq:two-traits-V-eq1}
gives us

$$
\begin{split}
    0 &= \alpha_0 ~ \Gamma ~ \text{e}^{-2 ~ V_{1}^2 } ~ V_{1}
        - f ~ ( V_{1} + \eta ~ V_{1} ) \\
    &= V_{1} \left[ \alpha_0 ~ \Gamma ~ \text{e}^{-2 ~ V_{1}^2 }
        - f ~ ( 1 + \eta ) \right]
    \text{.}
\end{split}
$$

One solution is $V_{1} = 0$, but if $V_{1} \ne 0$


\begin{equation}
\begin{split}
    \alpha_0 ~ \Gamma ~ \text{e}^{-2 ~ V_{1}^2 } &=
        f ~ ( 1 + \eta ) \\
    -2 ~ V_{1}^2 &=
        \log \left( \frac{ f ~ ( 1 + \eta ) }{ \alpha_0 ~ \Gamma } \right) \\
    V_{1} &= \pm \sqrt{\frac{1}{2}
        \log \left( \frac{ \alpha_0 ~ \Gamma }{ f ~ ( 1 + \eta ) } \right) }
    \text{.}
\end{split}
\label{eq:two-traits-V-eq5}
\end{equation}


When $V_{1} = - V_{2}$ and $V_{1} \ne 0$,

$$
    V_{1} = \pm \sqrt{\frac{1}{2}
        \log \left( \frac{ \alpha_0 ~ \Gamma }{ f ~ ( 1 - \eta ) } \right) }
    \text{.}
$$


When $\eta = 0$ and at least one of the two traits $\ne 0$,
traits are constrained by their distance
from the origin: $\sqrt{V_{1}^2 + V_{2}^2}$.
When $V_{1} \ne 0$, this distance is

$$
\begin{split}
    0 &= \alpha_0 ~ \Gamma ~
            \text{e}^{-V_{1}^2 - V_{2}^2} ~ V_{1}
        - f ~ V_{1} \\
    0 &= \alpha_0 ~ \Gamma ~
        \text{e}^{- ( V_{1}^2 + V_{2}^2) }
        - f \\
    \log \left( \frac{\alpha_0 ~ \Gamma}{ f } \right) &=
        V_{1}^2 + V_{2}^2 \\
     \sqrt{ V_{1}^2 + V_{2}^2 } &=
        \sqrt{ \log \left( \frac{\alpha_0 ~ \Gamma}{ f } \right)}
    \text{.}
\end{split}
$$

So the relationship between traits when $\eta = 0$ is

$$
    V_{1} =
    \pm \sqrt{
        \log \left( \frac{\alpha_0 ~ \Gamma}{ f } \right) -
        V_{2}^2
    }
    \text{,}
$$

when $V_{2}^2 \ge \log (\alpha_0 \Gamma / f)$.


## Summed abundances

Fitness for the species is written as

$$
    F = \exp \left\{
        r_0 - f ( V_{1}^2 + 2 \eta V_{1} V_{2} + V_{2}^2 ) -
        \alpha_0 \text{e}^{ - V_{1}^2 - V_{2}^2 } \Gamma
    \right\}
    \text{.}
$$


At equilibrium,

$$
    0 = r_0 - f ( V_{1}^2 + 2 \eta V_{1} V_{2} + V_{2}^2 ) -
        \alpha_0 \text{e}^{ - V_{1}^2 - V_{2}^2 } \Gamma
    \text{.}
$$


When $V_1 = V_2$, we can insert our answer from
\ref{eq:two-traits-V-eq5} to get

$$
\begin{split}
    0 &= r_0 - 2 ~ f ~ V_{1}^2 ( 1 + \eta ) -
        \alpha_0 \text{e}^{ -2 ~ V_{1}^2 } \Gamma \\
    r_0 &= 2 f ( 1 + \eta ) \left[
        \frac{1}{2}
        \log \left( \frac{ \alpha_0 ~ \Gamma }{ f ~ ( 1 + \eta ) } \right)
    \right] +
        \alpha_0 \text{e}^{ -2 ~
            \left[
                \frac{1}{2} \log \left(
                    \frac{ \alpha_0 ~ \Gamma }{ f ~ ( 1 + \eta ) }
                \right)
            \right]
        } \Gamma \\
    r_0 &= f ( 1 + \eta ) \log \left(
        \frac{ \alpha_0 ~ \Gamma }{ f ~ ( 1 + \eta ) }
    \right) + f ( 1 + \eta ) \\
    \frac{  r_0 - f ( 1 + \eta ) }{ f ( 1 + \eta ) } &=
        \log \left(
        \frac{ \alpha_0 ~ \Gamma }{ f ~ ( 1 + \eta ) }
        \right) \\
    \Gamma &= \frac{ f ~ ( 1 + \eta ) }{ \alpha_0 } ~
        \text{e}^{\frac{  r_0 - f ( 1 + \eta ) }{ f ( 1 + \eta ) } }
    \text{.}
\end{split}
$$


When $V_1 = - V_2$, we get

$$
    \Gamma = \frac{ f ~ ( 1 - \eta ) }{ \alpha_0 } ~
        \text{e}^{\frac{  r_0 - f ( 1 - \eta ) }{ f ( 1 - \eta ) } }
    \text{.}
$$


When $\eta = 0$,

$$
    \Gamma = \frac{ f }{ \alpha_0 } ~ \text{e}^{\frac{  r_0 - f }{ f } }
    \text{.}
$$


## Abundances among species 

Below is the equilibrium abundances
for any two ($i$ and $j$) of the $n$ species.
For simplicity, $X \equiv V_1^2 + V_2^2$ for a given scenario


$$
\begin{split}
    \Gamma &= N_i + \text{e}^{-d X} N_j +
        \text{e}^{-d X} \sum_{k \ne i,j}^{n}{ N_k } \\
    \Gamma &= N_j + \text{e}^{-d X} N_i +
        \text{e}^{-d X} \sum_{k \ne i,j}^{n}{ N_k }
    \text{.}
\end{split}
$$

Now we compare them:

$$
\begin{split}
    N_i + \text{e}^{-d X} N_j &=
        N_j + \text{e}^{-d X} N_i \\
    N_i \left( 1 - \text{e}^{-d X} \right) &=
        N_j \left( 1 - \text{e}^{-d X} \right) \\
    N_i &= N_j
    \text{.}
\end{split}
$$

So equilibrium abundances do not differ among species.




## Final 2-trait solutions

Combining the trait solutions with the abundance solutions produces
the following solutions.

When $V1 = V2$,

$$
\begin{split}
    \Gamma &= \frac{ f ~ ( 1 + \eta ) }{ \alpha_0 } ~
        \text{e}^{\frac{  r_0 - f ( 1 + \eta ) }{ f ( 1 + \eta ) } }
        \\
    V_1 &= \pm \sqrt{
        \frac{ r_0 - f (1 + \eta) }{ 2 f (1 + \eta) }
    }
    \text{.}
\end{split}
$$


When $V1 = - V2$,

$$
\begin{split}
    \Gamma &= \frac{ f ~ ( 1 - \eta ) }{ \alpha_0 } ~
        \text{e}^{\frac{  r_0 - f ( 1 - \eta ) }{ f ( 1 - \eta ) } }
        \\
    V_1 &= \pm \sqrt{
        \frac{ r_0 - f (1 - \eta) }{ 2 f (1 - \eta) }
    }
    \text{.}
\end{split}
$$

When $\eta = 0$,

$$
\begin{split}
    \Gamma &= \frac{ f }{ \alpha_0 } ~ \text{e}^{\frac{  r_0 - f }{ f } } \\
    \sqrt{V_1^2 + V_2^2} &= \sqrt{ \frac{ r_0 - f }{ f } } \\
    V_1 &= \sqrt{ \frac{ r_0 - f }{ f } - V_2^2 }
    \text{.}
\end{split}
$$



